<a href="https://colab.research.google.com/github/mr-ayush-agrawal/DL_Notebooks/blob/main/Lect/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transfer Larning
Useing the Transfer learning of Model VGG16 for the purpose of making a cat-vs-dog classifier which we made in some lecture before.

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'dogs-vs-cats:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F84954%2F196452%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240720%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240720T024334Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D14367a2473f1917c19680b1239b286b6519017c59f350e259d210bdcf3fd46c679cd981f13c635ea6e5b5ce76978d73059e15cc527754bd0341cafa83a893045887b8b740f809a4694d860c8df060a13f966a18cb87ec5b9a88750f90d89a9014624de46c8a8e275fef96bce9a4174111383c5dd8f0aabdd2dec0d5adc627d7ce31a526eec67d533eb86a87d02e29045a5a7733ecc6b3aa500032ffdc9c2ab7d2caf5f593044915ae1a4cd2248d9533aaf3f5271106e658d129e9f2ab89d1f3303132c023a9a34f496f167fc9fa6d47d99395f1f3497e992e76c9e98e0cc652e08929268395f693c913765fec9ffc759306b1ed4797fbcf8f1a60f232078e07f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 1142891694 bytes downloaded
Downloaded and uncompressed: dogs-vs-cats
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
count = 0
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        count += 1
print(count)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Streaming output truncated to the last 5000 lines.
/kaggle/input/dogs-vs-cats/train/dogs/dog.2166.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.780.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.9064.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.299.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.5971.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.3795.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.332.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.5733.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.3036.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.4157.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.6717.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.9095.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.4621.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.1939.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.3368.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.7858.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.11222.jpg
/kaggle/input/dogs-vs-cats/train/dogs/dog.9979.jpg
/kaggle/input/dogs-vs-cats/train/

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten
from keras.applications.vgg16 import VGG16

In [4]:
conv_base = VGG16(
    weights= 'imagenet',
    include_top = False,          # Removes the final Dense layers
    input_shape=(150, 150, 3)
)

58889256/58889256 [==============================] - 0s 0us/step


In [5]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [6]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation= 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16812353 (64.13 MB)
Trainable params: 16812353 (64.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# Freezing the Conv base layer
conv_base.trainable = False

In [9]:
# Model after freezing
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16812353 (64.13 MB)
Trainable params: 2097665 (8.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


Now all the paramertes of the Convolution layer has been converted to Non-trainable parameters

In [10]:
# Generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/dogs-vs-cats/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 80,
    image_size = (150, 150)
)

val_ds = keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/dogs-vs-cats/test',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 20,
    image_size = (150, 150)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [11]:
# Normalizing the data
def normalize_image(image, label):
    image = tf.cast(image/255, tf.float32)
    return image, label

train_ds = train_ds.map(normalize_image)
val_ds = val_ds.map(normalize_image)

In [12]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [13]:
%%time

history = model.fit(
    train_ds,
    epochs = 10,
    validation_data = val_ds
)

Epoch 1/10
250/250 [==============================] - 72s 240ms/step - loss: 0.2655 - accuracy: 0.8873 - val_loss: 0.2314 - val_accuracy: 0.9008
Epoch 2/10
250/250 [==============================] - 57s 226ms/step - loss: 0.1924 - accuracy: 0.9197 - val_loss: 0.2050 - val_accuracy: 0.9136
Epoch 3/10
250/250 [==============================] - 65s 256ms/step - loss: 0.1523 - accuracy: 0.9380 - val_loss: 0.2548 - val_accuracy: 0.8996
Epoch 4/10
250/250 [==============================] - 59s 230ms/step - loss: 0.1468 - accuracy: 0.9395 - val_loss: 0.2650 - val_accuracy: 0.8954
Epoch 5/10
250/250 [==============================] - 63s 251ms/step - loss: 0.1138 - accuracy: 0.9554 - val_loss: 0.2538 - val_accuracy: 0.9038
Epoch 6/10
250/250 [==============================] - 60s 232ms/step - loss: 0.0889 - accuracy: 0.9667 - val_loss: 0.2930 - val_accuracy: 0.8982
Epoch 7/10
250/250 [==============================] - 64s 254ms/step - loss: 0.0629 - accuracy: 0.9781 - val_loss: 0.3055 - val_ac

### Data Augumentation

In [15]:
from keras.preprocessing.image import ImageDataGenerator

In [16]:
train_datagen = ImageDataGenerator(
    rescale= 1.0/255,
    shear_range = 0.2,
    zoom_range  = 0.3,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_gen = train_datagen.flow_from_directory(
    '/kaggle/input/dogs-vs-cats/train',
    target_size = (150, 150),
    batch_size = 80,
    class_mode = 'binary'
)

val_gen = test_datagen.flow_from_directory(
    '/kaggle/input/dogs-vs-cats/test',
    target_size = (150, 150),
    batch_size = 20,
    class_mode = 'binary'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [17]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [18]:
%%time

history = model.fit_generator(
    train_gen,
    epochs = 10,
    validation_data= val_gen
)

<timed eval>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
250/250 [==============================] - 151s 599ms/step - loss: 0.2330 - accuracy: 0.9070 - val_loss: 0.2214 - val_accuracy: 0.9096
Epoch 2/10
250/250 [==============================] - 147s 588ms/step - loss: 0.1987 - accuracy: 0.9176 - val_loss: 0.2019 - val_accuracy: 0.9180
Epoch 3/10
250/250 [==============================] - 156s 626ms/step - loss: 0.1884 - accuracy: 0.9218 - val_loss: 0.2004 - val_accuracy: 0.9164
Epoch 4/10
250/250 [==============================] - 150s 599ms/step - loss: 0.1884 - accuracy: 0.9194 - val_loss: 0.2090 - val_accuracy: 0.9180
Epoch 5/10
250/250 [==============================] - 155s 618ms/step - loss: 0.1746 - accuracy: 0.9250 - val_loss: 0.2060 - val_accuracy: 0.9164
Epoch 6/10
250/250 [==============================] - 151s 604ms/step - loss: 0.1672 - accuracy: 0.9312 - val_loss: 0.1969 - val_accuracy: 0.9188
Epoch 7/10
250/250 [==============================] - 147s 589ms/step - loss: 0.1670 - accuracy: 0.9295 - val_loss: 0.1976 -

Not much but there has been increase in the accuracy of the model

## Fine Tuning Transfer Model

Using the fine tuning approach to make the mdoel

In [21]:
# using the same conv_base defined above
conv_base

In [23]:
# Making the pareameters true
conv_base.trainable = True

set_trainable = False

for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else :
    layer.trainable = False

for layer in conv_base.layers:
  print(layer.name, layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


Only Last few are trainable and rest won't train while making the model

In [24]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

We can note that half of the parameters are not getting trained.

In [26]:
fine_model = Sequential ()

fine_model.add(conv_base)
fine_model.add(Flatten())
fine_model.add(Dense(256, activation = 'relu'))
fine_model.add(Dense(1, activation = 'sigmoid'))

From Here the code is same as of the Feature extraction model.